In [72]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
import os
import unittest


### Path To Dataset



In [73]:
dataset_path = 'data/brain-mri-images/brain_tumor_dataset'

### Loading Dataset Images

In [74]:
def load_images(data_path, img_size=(128, 128)):
    images = []
    labels = []

    for label, class_name in enumerate(['no', 'yes']):
        class_path = os.path.join(data_path, class_name)
        for file in os.listdir(class_path):
            img_path = os.path.join(class_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

X,y = load_images(dataset_path)
X = X / 255.0
X = X.reshape(-1,128,128,1)


    

## Preproccesing Data

In [75]:

def train_test_split_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split_data(X, y)


## Initializing Network Parameters

In [76]:
def initialize_parameters():
    parameters = {
        "W1": np.random.randn(3, 3, 1, 32) * 0.01,
        "b1": np.zeros((1, 1, 1, 32)),
        "W2": np.random.randn(3, 3, 32, 64) * 0.01,
        "b2": np.zeros((1, 1, 1, 64)),
        "W3": np.random.randn(128, 64 * 6 * 6) * 0.01,
        "b3": np.zeros((128, 1)),
        "W4": np.random.randn(1, 128) * 0.01,
        "b4": np.zeros((1, 1))
    }
    return parameters



## Helper Functions

In [ ]:
def relu(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

errorMessage = "Invalid dimensions for convolution. Check the padding and stride values."

def conv_forward(A_prev, W, b, stride=1, pad=0):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape

    n_H = int((n_H_prev - f + 2 * pad) / stride) + 1
    n_W = int((n_W_prev - f + 2 * pad) / stride) + 1

    if n_H <= 0 or n_W <= 0:
        raise ValueError(errorMessage)

    Z = np.zeros((m, n_H, n_W, n_C))

    A_prev_pad = np.pad(A_prev, ((0, 0), (pad, pad), (pad, pad), (0, 0)),
                        mode='constant', constant_values=(0, 0))

    for i in range(m):
        a_prev_pad = A_prev_pad[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    Z[i, h, w, c] = np.sum(a_slice_prev * W[:, :, :, c]) + b[:, :, :, c].item()

    return Z

def pool_forward(A_prev, f=2, stride=2, mode='max'):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev

    if n_H <= 0 or n_W <= 0:
        raise ValueError(errorMessage)

    A = np.zeros((m, n_H, n_W, n_C))

    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    a_slice_prev = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]

                    if mode == 'max':
                        A[i, h, w, c] = np.max(a_slice_prev)
                    elif mode == 'average':
                        A[i, h, w, c] = np.mean(a_slice_prev)

    return A

def flatten(A):
    return A.reshape(A.shape[0], -1)

def dense_forward(A_prev, W, b):
    Z = np.dot(W, A_prev) + b
    return Z


def conv_backward(dZ, A_prev, W, b, stride=1, pad=0):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape

    (m, n_H, n_W, n_C) = dZ.shape

    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    A_prev_pad = np.pad(A_prev, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode='constant', constant_values=(0, 0))
    dA_prev_pad = np.pad(dA_prev, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode='constant', constant_values=(0, 0))

    for i in range(m):
        a_prev_pad = A_prev_pad[i]
        da_prev_pad = dA_prev_pad[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:, :, :, c] * dZ[i, h, w, c]
                    dW[:, :, :, c] += a_slice * dZ[i, h, w, c]
                    db[:, :, :, c] += dZ[i, h, w, c]

        if pad != 0:
            dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]
        else:
            dA_prev[i, :, :, :] = da_prev_pad

    return dA_prev, dW, db

def pool_backward(dA, A_prev, f=2, stride=2, mode='max'):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (m, n_H, n_W, n_C) = dA.shape

    dA_prev = np.zeros(A_prev.shape)

    for i in range(m):
        a_prev = A_prev[i]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    if mode == 'max':
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                        mask = (a_prev_slice == np.max(a_prev_slice))
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += mask * dA[i, h, w, c]
                    elif mode == 'average':
                        da = dA[i, h, w, c]
                        shape = (f, f)
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += da / (f * f)

    return dA_prev




## Propagation

In [ ]:
def forward_propagation(X, parameters):
    W1, b1 = parameters["W1"], parameters["b1"]
    W2, b2 = parameters["W2"], parameters["b2"]
    W3, b3 = parameters["W3"], parameters["b3"]
    W4, b4 = parameters["W4"], parameters["b4"]

    Z1 = conv_forward(X, W1, b1)
    A1 = relu(Z1)
    P1 = pool_forward(A1)

    Z2 = conv_forward(P1, W2, b2)
    A2 = relu(Z2)
    P2 = pool_forward(A2)

    F = flatten(P2)

    Z3 = dense_forward(F, W3, b3)
    A3 = relu(Z3)

    Z4 = dense_forward(A3, W4, b4)
    A4 = sigmoid(Z4)

    cache = {
        "Z1": Z1, "A1": A1, "P1": P1,
        "Z2": Z2, "A2": A2, "P2": P2,
        "F": F, "Z3": Z3, "A3": A3,
        "Z4": Z4, "A4": A4
    }

    return A4, cache

forward_propagation(X,initialize_parameters())


def backward_propagation(X, Y, parameters, cache):
    W1, b1 = parameters["W1"], parameters["b1"]
    W2, b2 = parameters["W2"], parameters["b2"]
    W3, b3 = parameters["W3"], parameters["b3"]
    W4, b4 = parameters["W4"], parameters["b4"]

    Z1, A1, P1, Z2, A2, P2 = cache["Z1"], cache["A1"], cache["P1"], cache["Z2"], cache["A2"], cache["P2"]
    F, Z3, A3, Z4, A4 = cache["F"], cache["Z3"], cache["A3"], cache["Z4"], cache["A4"]
    m = X.shape[0]

    dA4 = -(np.divide(Y, A4) - np.divide(1 - Y, 1 - A4))

    dZ4 = dA4 * A4 * (1 - A4)
    dW4 = np.dot(dZ4, A3.T) / m
    db4 = np.sum(dZ4, axis=1, keepdims=True) / m

    dA3 = np.dot(W4.T, dZ4)
    dZ3 = dA3 * (A3 > 0)
    dW3 = np.dot(dZ3, F.T) / m
    db3 = np.sum(dZ3, axis=1, keepdims=True) / m

    dP2 = np.dot(W3.T, dZ3)
    dP2 = dP2.reshape(P2.shape)

    dA2 = pool_backward(dP2, A2, mode='max')
    dZ2 = dA2 * (A2 > 0)
    dA1, dW2, db2 = conv_backward(dZ2, P1, W2, b2, stride=1, pad=0)

    dP1 = pool_backward(dA1, A1, mode='max')

    dZ1 = dP1 * (A1 > 0)
    dA0, dW1, db1 = conv_backward(dZ1, X, W1, b1, stride=1, pad=0)

    grads = {
        "dW1": dW1, "db1": db1,
        "dW2": dW2, "db2": db2,
        "dW3": dW3, "db3": db3,
        "dW4": dW4, "db4": db4
    }

    return grads

## Update Parameters

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    parameters["W1"] -= learning_rate * grads["dW1"]
    parameters["b1"] -= learning_rate * grads["db1"]
    parameters["W2"] -= learning_rate * grads["dW2"]
    parameters["b2"] -= learning_rate * grads["db2"]
    parameters["W3"] -= learning_rate * grads["dW3"]
    parameters["b3"] -= learning_rate * grads["db3"]
    parameters["W4"] -= learning_rate * grads["dW4"]
    parameters["b4"] -= learning_rate * grads["db4"]

    return parameters


## Compute Loss

In [ ]:

def compute_loss(A4, Y):
    m = Y.shape[0]
    cost = -1/m * np.sum(Y * np.log(A4) + (1 - Y) * np.log(1 - A4))
    cost = np.squeeze(cost)
    return cost

## Tests

In [ ]:
class TestModel(unittest.TestCase):

    def setUp(self):
        self.parameters = initialize_parameters()
        self.X_train = np.random.randn(10, 128, 128, 1)  
        self.Y_train = np.random.randint(0, 2, (10, 1))  
        self.X_test = np.random.randn(5, 128, 128, 1)    
        self.Y_test = np.random.randint(0, 2, (5, 1))    
        self.num_epochs = 10
        self.learning_rate = 0.01

    def test_initialize_parameters(self):
        parameters = initialize_parameters()
        self.assertEqual(parameters["W1"].shape, (3, 3, 1, 32))
        self.assertEqual(parameters["b1"].shape, (1, 1, 1, 32))
        self.assertEqual(parameters["W2"].shape, (3, 3, 32, 64))
        self.assertEqual(parameters["b2"].shape, (1, 1, 1, 64))
        self.assertEqual(parameters["W3"].shape, (128, 64 * 6 * 6))
        self.assertEqual(parameters["b3"].shape, (128, 1))
        self.assertEqual(parameters["W4"].shape, (1, 128))
        self.assertEqual(parameters["b4"].shape, (1, 1))

    def test_forward_propagation(self):
        A4, cache = forward_propagation(self.X_train, self.parameters)
        self.assertEqual(A4.shape, (1, self.X_train.shape[0]))

    def test_backward_propagation(self):
        A4, cache = forward_propagation(self.X_train, self.parameters)
        grads = backward_propagation(self.X_train, self.Y_train, self.parameters, cache)
        self.assertEqual(grads["dW1"].shape, self.parameters["W1"].shape)
        self.assertEqual(grads["db1"].shape, self.parameters["b1"].shape)
        self.assertEqual(grads["dW2"].shape, self.parameters["W2"].shape)
        self.assertEqual(grads["db2"].shape, self.parameters["b2"].shape)
        self.assertEqual(grads["dW3"].shape, self.parameters["W3"].shape)
        self.assertEqual(grads["db3"].shape, self.parameters["b3"].shape)
        self.assertEqual(grads["dW4"].shape, self.parameters["W4"].shape)
        self.assertEqual(grads["db4"].shape, self.parameters["b4"].shape)

    def test_update_parameters(self):
        A4, cache = forward_propagation(self.X_train, self.parameters)
        grads = backward_propagation(self.X_train, self.Y_train, self.parameters, cache)
        updated_parameters = update_parameters(self.parameters, grads, self.learning_rate)
        self.assertEqual(updated_parameters["W1"].shape, self.parameters["W1"].shape)
        self.assertEqual(updated_parameters["b1"].shape, self.parameters["b1"].shape)
        self.assertEqual(updated_parameters["W2"].shape, self.parameters["W2"].shape)
        self.assertEqual(updated_parameters["b2"].shape, self.parameters["b2"].shape)
        self.assertEqual(updated_parameters["W3"].shape, self.parameters["W3"].shape)
        self.assertEqual(updated_parameters["b3"].shape, self.parameters["b3"].shape)
        self.assertEqual(updated_parameters["W4"].shape, self.parameters["W4"].shape)
        self.assertEqual(updated_parameters["b4"].shape, self.parameters["b4"].shape)

    def test_compute_loss(self):
        A4, _ = forward_propagation(self.X_train, self.parameters)
        cost = compute_loss(A4, self.Y_train)
        self.assertIsInstance(cost, float)

if __name__=='__main__':
    unittest.main(argv=[''], exit=False)

/var/folders/wd/t26gn6gx5rz1zw4wphrdt0km0000gn/T/ipykernel_28987/1495913819.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z[i, h, w, c] = np.sum(a_slice_prev * W[:, :, :, c]) + float(b[:, :, :, c])
EE.E
ERROR: test_backward_propagation (__main__.TestModel.test_backward_propagation)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/wd/t26gn6gx5rz1zw4wphrdt0km0000gn/T/ipykernel_28987/2043013296.py", line 28, in test_backward_propagation
    A4, cache = forward_propagation(self.X_train, self.parameters)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/wd/t26gn6gx5rz1zw4wphrdt0km0000gn/T/ipykernel_28987/3729796183.py", line 17, in forward_propagation
    Z3 = dense_forward(F, W3, b3)
         ^^

## Train Model

In [ ]:

def train_model(X_train, Y_train, parameters, num_epochs, learning_rate):
    costs = []
    accuracies = []

    for epoch in range(num_epochs):
        A4, cache = forward_propagation(X_train, parameters)
        cost = compute_loss(A4, Y_train)
        grads = backward_propagation(X_train, Y_train, parameters, cache)
        parameters = update_parameters(parameters, grads, learning_rate)
        
        predictions = (A4 > 0.5).astype(int)
        accuracy = np.mean(predictions == Y_train)
        
        if epoch % 100 == 0:
            print(f"Cost after epoch {epoch}: {cost}, Accuracy: {accuracy}")
            costs.append(cost)
            accuracies.append(accuracy)
    
    return parameters, costs, accuracies


## Evaluation

In [ ]:
def evaluate_accuracy(X, Y, parameters):
    A4, _ = forward_propagation(X, parameters)
    predictions = (A4 > 0.5).astype(int)
    accuracy = np.mean(predictions == Y)
    return accuracy


## Plots

In [ ]:
parameters = initialize_parameters()
num_epochs = 1000
learning_rate = 0.01
parameters, costs, accuracies = train_model(X_train, y_train, parameters, num_epochs, learning_rate)
test_accuracy = evaluate_accuracy(X_test, y_test, parameters)

plt.subplot(1, 2, 1)
plt.plot(costs)
plt.ylabel('Cost')
plt.xlabel('Epoch (per 100)')
plt.title('Cost reduction over time')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy')
plt.xlabel('Epoch (per 100)')
plt.title('Accuracy over time')

plt.show()


print(f"Test Accuracy: {test_accuracy}")

/var/folders/wd/t26gn6gx5rz1zw4wphrdt0km0000gn/T/ipykernel_28987/1495913819.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z[i, h, w, c] = np.sum(a_slice_prev * W[:, :, :, c]) + float(b[:, :, :, c])


ValueError: shapes (128,2304) and (202,57600) not aligned: 2304 (dim 1) != 202 (dim 0)